In [23]:
import vrep 
import sys
import time 
import numpy as np
import skfuzzy as fuzz

from tank import *

In [24]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [25]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [26]:
tank.forward(5)

#continue reading and printing values from proximity sensors
t = time.time()
counter = 0
while (time.time()-t)<15: # read values for 5 seconds
    counter +=1

    for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
        if(err_code == 0):
            if counter % 5000 == 0:
                print("Proximity_sensor_"+sensor_name, np.linalg.norm(detectedPoint))
                print()
    # print()
    
tank.stop()

Proximity_sensor_EN 1.1104384546986024

Proximity_sensor_ES 1.1104384546986024

Proximity_sensor_NE 1.4395078823411438

Proximity_sensor_NW 2.6980321106366065

Proximity_sensor_SE 2.6980321106366065

Proximity_sensor_SW 2.6980321106366065

Proximity_sensor_WN 2.509908169606907

Proximity_sensor_WS 2.509908169606907

Proximity_sensor_EN 1.1054168887585394

Proximity_sensor_ES 1.1054168887585394

Proximity_sensor_NE 1.4847513206245488

Proximity_sensor_NW 2.6619472593061473

Proximity_sensor_SE 2.6619472593061473

Proximity_sensor_SW 2.6619472593061473

Proximity_sensor_WN 2.4779202769529043

Proximity_sensor_WS 2.4779202769529043

Proximity_sensor_EN 1.1004176772717582

Proximity_sensor_ES 1.1004176772717582

Proximity_sensor_NE 1.550161859198238

Proximity_sensor_NW 2.9452489644102684

Proximity_sensor_SE 2.9452489644102684

Proximity_sensor_SW 2.9452489644102684

Proximity_sensor_WN 2.430906786697502

Proximity_sensor_WS 2.430906786697502

Proximity_sensor_EN 1.0436454722484387

Proxi